# Running Semantic Functions Inline

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your C# code. This can be useful in a few scenarios:

* Dynamically generating the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating. 

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the kernel arguments.

Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Let's create a semantic function used to summarize content:

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

This allows to see the prompt before it's sent to AI.

In [ ]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);

Let's transform the prompt template into a function that the kernel can execute:

In [ ]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

In [ ]:
var input = """
Welcome to Live! 360 Events
Your Ultimate Education Destination
Live! 360® Conferences and Seminars are for Enterprise Developers, IT Implementors, 
IT Decision Makers, IT Security Teams, and Data Professionals looking to explore both 
current and leading-edge technologies, and to broaden their horizons on what lies ahead 
in the ever-evolving tech landscape.
Join us for our annual flagship event, Live! 360 Tech Con, a 6-in-1 conference in 
Orlando this November, where industry leaders and IT professionals converge to train, 
network, collaborate, and unlock the full potential of enterprise technology. 
Experience the synergy of our diverse learning tracks, world-class instructors and speakers,
and hands-on labs and workshops. Secure your spot now and be part of Live! 360 2024 - 
where education meets innovation. 
""";

...and run the summary function:

In [ ]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summaryResult);

The code above shows all the steps, to understand how the function is composed step by step. However, the kernel
includes also some helpers to achieve the same more concisely.

The same function above can be executed with less code:

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text.




In [ ]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) Live!360 Events are your Ultimate Education Destination

    2) Live! 360® Conferences and Seminars are for Enterprise Developers, IT Implementors, IT Decision Makers, IT Security Teams, and Data Professionals.

    3) Data Professionals looking to explore both current and leading-edge technologies, and to broaden their horizons on what lies ahead in the ever-evolving tech landscape.
";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = textToSummarize });

Console.WriteLine(result);